In [1]:
require(tidyverse)
require(data.table)

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.1  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [2]:
sscore_f <- '/oak/stanford/groups/mrivas/projects/PRS/private_output/snpnet_PRS/HC_20190303/8_score_all/HC382.sscore'
sscore_df <- fread(
    cmd=paste0('cat ', sscore_f, ' | cut -f2,6'), sep='\t', data.table=F
)


In [5]:
phe_f <- '/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/extras/highconfidenceqc/phe/HC382.phe'
phe_df <- fread(
    cmd=paste0('cat ', phe_f, ' | cut -f2,3'), sep='\t', data.table=F
)
colnames(phe_df) <- c('IID', 'HC382')


In [31]:
combined_df <- sscore_df %>% rename(snpnet_PRS = SCORE1_AVG) %>% 
left_join(phe_df, by='IID') %>%
rename(outcome = HC382) %>% 
filter(outcome != -9) %>%
drop_na() %>%
mutate(Percentile = rank(-snpnet_PRS) / n())


In [32]:
combined_df %>% dim()

[1] 488374      4

In [34]:
cnt_middle <- combined_df %>% 
filter(0.4 <= Percentile, Percentile <= 0.6) %>% 
count(outcome) %>% rename(n_40_60 = n)

In [35]:
cnt_middle

outcome,n_40_60
1,87152
2,10522


In [45]:
compute_or <- function(df){
    joined_df <- df %>% count(outcome) %>% inner_join(cnt_middle, by='outcome') %>% gather(bin, cnt, -outcome)
    n_TP <- joined_df %>% filter(bin == 'n', outcome == 2) %>% select(cnt) %>% pull()
    n_TN <- joined_df %>% filter(bin != 'n', outcome == 1) %>% select(cnt) %>% pull()
    n_FP <- joined_df %>% filter(bin != 'n', outcome == 2) %>% select(cnt) %>% pull()
    n_FN <- joined_df %>% filter(bin == 'n', outcome == 1) %>% select(cnt) %>% pull()
    or <- (n_TP * n_TN) / (n_FP * n_FN)
    lor <- log(or)
    se_lor <- sqrt((1/n_TP) + (1/n_TN) + (1/n_FP) + (1/n_FN))
    lor <- exp(lor - 1.96 * se_lor)
    uor <- exp(lor + 1.96 * se_lor)
    c(or, lor, uor) 
#     %>% print()    
}

In [48]:
idx * 0.Percentile == 1
combined_df %>% filter(idx * 0.Percentile == 1) 

IID,snpnet_PRS,outcome,Percentile
4237947,-0.00014146,1,1


In [51]:
compute_or_wrapper <- function(idx, df){
    bin_l = idx * 0.1
    bin_u = (idx + 1) * 0.1
    c(
        bin_l, bin_u, 
        df %>% filter(bin_l < Percentile, Percentile <= bin_u) 
        %>% compute_or()
    )    
}

In [57]:
res <- lapply(0:9, compute_or_wrapper, combined_df)

In [62]:
dff <- as.data.frame(res) 
colnames(dff) <- lapply(0:9, function(x){x * 10 + 5})


In [65]:
rownames(dff) <- c(
    'l', 'u', 'OR', 'OR_l', 'OR_u'
)

In [66]:
dff

,5,15,25,35,45,55,65,75,85,95
l,0.000000,0.100000,0.200000,0.300000,0.400000,0.5000000,0.6000000,0.7000000,0.8000000,0.900000
u,0.100000,0.200000,0.300000,0.400000,0.500000,0.6000000,0.7000000,0.8000000,0.9000000,1.000000
OR,3.757193,2.073254,1.576222,1.259013,1.085349,0.9161921,0.8041877,0.6944641,0.5670122,0.435018
OR_l,3.654001,2.011964,1.527283,1.218016,1.048743,0.8839145,0.7748165,0.6679848,0.5439712,0.415677
OR_u,39.719815,7.705791,4.753225,3.494257,2.953681,2.5087389,2.2524602,2.0276143,1.7958092,1.585906


In [68]:
dff %>% fwrite('HC382.csv', row.names = T, col.names=T)